In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

In [9]:
#read data
files = ['America_nonNeutral','China_nonNeutral','americaData', 'AppleData', 'chinaData', 'ISIS_Data','ObamaData','RepublicansData','SandersData','TeslaData','WindowsData'];

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

def noNeutrals(dataframe):
    data=dataframe.overallPol
    drop = np.array([i for i in range(len(dataframe.body)) if(data[i]==0)])
    newDf = dataframe.drop(drop)
    newDf.reset_index(drop=True,inplace=True)#resets index
    return newDf

def readData(file_name):
    dataFrame = pickle.load(open(file_name + ".pickle",'rt'))
    #dataFrame.columns = ['body', 'score', 'overallpol','stdPol', 'overallSub', 'stdSub', 'polRange',>
    dataFrame = noNeutrals(dataFrame)
    y = dataFrame['score'].values
    x = dataFrame[['overallPol', 'stdSub', 'overallSub', 'sentLen']].values
    return y, x
# select non_neutral data from files.
j=1

y, x = readData(files[j])

y_np = np.array(y)
x_np = np.array(x)
# standard normalization
from sklearn.preprocessing import StandardScaler
scy = StandardScaler()
scx = StandardScaler()
scy = scy.fit(y)
scx = scx.fit(x)
X = scx.transform(x)
Y = scy.transform(y)
#X.reshape(-1, 1)
#Y.reshape(-1, 1)
#X = preprocessing.scale(x_np)
#Y = preprocessing.scale(y_np)
######################################
#setup the dataset (supervised classification training) for neural network
######################################
from pybrain.utilities import percentError
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.structure.modules import SoftmaxLayer
from pybrain.datasets.classification import ClassificationDataSet
from pybrain.datasets import SupervisedDataSet

ds = SupervisedDataSet(4, 1)
for i in range(len(X)):
    ds.addSample(X[i], Y[i])
# #split the dataset
trainData, testData = ds.splitWithProportion(0.60)

# ###################################
# #Creating a Neural Network
# ###################################
# # build nerual net with 4 inputs, 5 hidden neuron and 1 output neuron
net = buildNetwork(4,3,1,bias=True)
trainer = BackpropTrainer(net, trainData, learningrate=0.0001, momentum=0.1)

train_non_RMS=[]
test_non_RMS=[]
numEpochs = [i*1 for i in range(20)]

train_out = net.activateOnDataset(trainData)
train_mse = mean_squared_error(trainData['target'], train_out)
test_out = net.activateOnDataset(testData)
test_mse = mean_squared_error(testData['target'], test_out)

train_non_RMS.append(train_mse)
test_non_RMS.append(test_mse)

for i in range(19):
    train_error = trainer.trainEpochs(1)

    train_out = net.activateOnDataset(trainData)
    train_mse = mean_squared_error(trainData['target'], train_out)
    test_out = net.activateOnDataset(testData)
    test_mse = mean_squared_error(testData['target'], test_out)

    train_non_RMS.append(train_mse)
    test_non_RMS.append(test_mse)

    
#Select dataset which within neutral data
j=3

y, x = readData(files[j])

y_np = np.array(y)
x_np = np.array(x)
# standard normalization
from sklearn.preprocessing import StandardScaler
scy = StandardScaler()
scx = StandardScaler()
scy = scy.fit(y)
scx = scx.fit(x)
X = scx.transform(x)
Y = scy.transform(y)
######################################
#setup the dataset (supervised classification training) for neural network
######################################
from pybrain.utilities import percentError
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.structure.modules import SoftmaxLayer
from pybrain.datasets.classification import ClassificationDataSet
from pybrain.datasets import SupervisedDataSet

ds = SupervisedDataSet(4, 1)
for i in range(len(X)):
    ds.addSample(X[i], Y[i])
# #split the dataset
trainData, testData = ds.splitWithProportion(0.60)

# ###################################
# #Creating a Neural Network
# ###################################
# # build nerual net with 4 inputs, 5 hidden neuron and 1 output neuron
net = buildNetwork(4,3,1,bias=True)
trainer = BackpropTrainer(net, trainData, learningrate=0.0001, momentum=0.1)

trainRMS=[]
testRMS=[]
numEpochs = [i*1 for i in range(20)]

train_out = net.activateOnDataset(trainData)
train_mse = mean_squared_error(trainData['target'], train_out)
test_out = net.activateOnDataset(testData)
test_mse = mean_squared_error(testData['target'], test_out)

trainRMS.append(train_mse)
testRMS.append(test_mse)

for i in range(19):
    train_error = trainer.trainEpochs(1)

    train_out = net.activateOnDataset(trainData)
    train_mse = mean_squared_error(trainData['target'], train_out)
    test_out = net.activateOnDataset(testData)
    test_mse = mean_squared_error(testData['target'], test_out)

    trainRMS.append(train_mse)
    testRMS.append(test_mse)

#plot the result  
plt.plot(numEpochs,trainRMS,label='train')
plt.plot(numEpochs,testRMS,label='test')
plt.plot(numEpochs,train_non_RMS,label='train_non')
plt.plot(numEpochs,test_non_RMS,label='test_non')

plt.xlabel('Number of Epochs')
plt.ylabel('MSE Error america nonneutral')
plt.legend()
plt.show()

/Users/Uvaguy/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/Uvaguy/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/Uvaguy/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/Uvaguy/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by StandardS